# Что будет
- сводные таблицы
- фильтры и вычисления с помощью метода loc
- фильтрация пустых значений через isnull
- время в pandas
- строковые методы
- немного про учет форм слов

# Сводные таблицы
Прям как в экселе

In [ ]:
import pandas as pd

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

### Вопрос аналитику
Какие оценки поставил пользователь №1 и в каком количестве?

Что сделаем:
1. Фильтруем датафрейм ratings для userId = 1
2. Считаем для этого пользователя сколько он выставил единиц, двоек итд

In [ ]:
ratings[ratings.userId == 1].rating.value_counts()

In [ ]:
ratings.groupby(['userId', 'rating']).count().head()

Как это сделать для всех пользователей сразу:

In [ ]:
ratings.pivot_table(index = ['userId', 'movieId'], columns = 'rating', values = 'timestamp', aggfunc = 'count', 
                    fill_value = 0).head()


In [ ]:
# можно итоги добавить
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).tail()

In [ ]:
pd.display.max.rows = 1000

In [ ]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).to_csv('pivot_table.csv')

In [ ]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).sort_values(5, ascending=False)[[2, 3.5]].head()

### Фильтры и вычисления с помощью loc и iloc

In [ ]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

In [ ]:
log[['user_id', 'region']].head()

In [ ]:
# метод loc позволяет выбрать строки и столбцы в соответствии с условиями
# двоеточие означает выбор всех значений

log.loc[:, ['user_id', 'region']].head()

In [ ]:
# iloc работает аналогично, но с указанием номера строки / столбца

log.iloc[:, [3, -1]].head()

In [ ]:
def vat(country):
    if country == 'Russia':
        return 1.2
    else:
        return None

In [ ]:
log['VAT'] = log.region.apply(vat)
log.head(10)

In [ ]:
# пример фильтра на страну
# если столбцы не надо фильтровать, то второй параметр можно не указывать

log.loc[log.region == 'Russia'].head()

In [ ]:
# пример вычисления нового столбца с НДС для страны

log.loc[log.region == 'Russia', 'VAT'] = 1.2
log.loc[log.region == 'China', 'VAT'] = 1.1
log.head(10)

In [ ]:
# вариант с вычисляемым фильтром

log.loc[lambda row: row.region == 'Russia'].head(10)

### Скорость метода loc
Посчитаем средний рейтинг разных жанров. Метод merge взят из материалов следующего занятия ¯ \ _ (ツ) _ / ¯

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')
joined.head()

In [ ]:
joined['Adventure'] = joined.apply(lambda row: row.rating if 'Adventure' in row.genres else None, axis=1)
joined.head()

Можно и через loc

In [ ]:
joined['Adventure'] = joined.loc[joined.genres.str.contains('Adventure'), 'rating']
joined.head()

Для замера времени сделаем эти операции для набор жанров

In [ ]:
genres = ['Adventure', 'Animation', 'Children', 'Drama', 'Musical', 'Thriller']

In [ ]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.apply(lambda row: row.rating if genre in row.genres else None, axis=1)

In [ ]:
joined.head()

In [ ]:
joined.Adventure.mean()

In [ ]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.loc[joined.genres.str.contains(genre), 'rating']

In [ ]:
for i range(len(joined)):
    joined.loc[i, ['']] = 

### Упражнение
Какие варианты источников трафика есть в столбце traffic_source?

Создайте столбец traffic_type, в котором для источников 'yandex' и 'google' будет стоять значение 'organic'. А для остальных вариантов - NaN.

In [ ]:
def log['timestamp']

In [ ]:
log.drop(columns=['timestamp'])

In [ ]:
log.loc[log.traffic_source == 'yandex' , 'traffic_type'] = 'organic' 
log.loc[log.traffic_source == 'google' , 'traffic_type'] = 'organic'
log.head()

In [ ]:
log.loc[(log.traffic_source == 'yandex') | (log.traffic_source == 'google'), 'traffic_type'] = 'organic' 

In [ ]:
log.loc[log.traffic_source.isin(['yandex', 'google']), 'traffic_type'] = 'organic'
log.head()

### Методы isnull, isna
Определение пустых или None значений. По сути одинаковые методы

In [ ]:
import numpy as np

In [ ]:
df = pd.DataFrame({'value': [123, None, np.nan, np.NaN, np.NAN, 456]})
df

In [ ]:
# фильтр на пустые значения в столбце value

df.loc[~pd.isnull(df.value), :]

In [ ]:
df.fillna(0)

In [ ]:
df.fillna(0).value.mean()

In [ ]:
df.value.mean()

### Упражнение
Для пустых значений в столбце traffic_type выставьте значение 'other'

### Дата и время в pandas

In [ ]:
log.head()

In [ ]:
log['date'] = pd.to_datetime(log['timestamp'], unit='s')
log.head()

In [ ]:
log['date'] = pd.to_datetime(log['timestamp'], format='%Y-%m-%dT%H:%M:%S')

In [ ]:
# столбец datetime64[ns] теперь имеет тип даты
log.info()

In [ ]:
# получим час визита

log['hour'] = log.date.dt.hour
log.head()

### Методы работы со строками

In [ ]:
stats = pd.read_csv('keywords.csv')
stats.head()

Проверка наличия подстроки в строке в питоне:

In [ ]:
'охотник' in 'каждый охотник желает знать...'

Аналог в pandas:

In [ ]:
stats[stats.keyword.str.contains('охотник')].head()

[Документация](https://www.geeksforgeeks.org/python-pandas-series-str-contains/)

Syntax: Series.str.contains(pat, case=True, flags=0, na=nan, regex=True)

Parameter :
- pat : Character sequence or regular expression.
- case : If True, case sensitive.
- flags : Flags to pass through to the re module, e.g. re.IGNORECASE.
- na : Fill value for missing values.
- regex : If True, assumes the pat is a regular expression.

In [ ]:
# поиск одного из нескольких слов

stats[stats.keyword.str.contains('охотник|фильм|2016')].head()

### Упражнение
Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

In [ ]:
stats[(stats.keyword.str.contains('погода в')) & (stats.keyword.str.contains('москв|новосибирск|краснодар'))]


### replace

In [ ]:
'отпуск начнется завтра'.replace('завтра', 'через месяц')

Аналог в pandas на запросах про сериалы:

In [ ]:
serial = stats[stats.keyword.str.contains('сериалы')]
serial.head()

In [ ]:
serial.keyword.str.replace('сериалы', 'книги').head()

### Как учитывать разное написание слов
Самое простое - методы upper и lower

In [ ]:
serial.keyword.str.upper().head()

In [ ]:
serial.keyword.str.lower().head()

### Что делать если нужно учесть формы написания слов?

In [ ]:
stats[stats.keyword.str.contains('рубл')].head()

Почему нельзя просто оставить str.contains('рубл'):

In [ ]:
non_financial_search = 'рубленая котлетка'

### Библиотека [pymystem](https://pypi.org/project/pymystem3/)

In [ ]:
from pymystem3 import Mystem

In [ ]:
search = 'курс гривны к рублю рубли рублях рублев рубленый культяпками однушками'

In [ ]:
m = Mystem()
lemmas = m.lemmatize(search)
lemmas

In [ ]:
' '.join(lemmas)

In [ ]:
стали  стать

In [ ]:
word2vec

## ========== Домашняя работа ==========

### Домашнее задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:
- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [1]:
import pandas as pd

log = pd.read_csv('visit_log.csv', sep=';')

log['source_type'] = log.loc[:, 'traffic_source']
log.loc[(log['traffic_source'] == 'yandex') | (log['traffic_source'] == 'google'), 'source_type'] = 'organic'
log.loc[(log['region'] == 'Russia') & (log['traffic_source'] == 'paid'), 'source_type'] = 'ad'
log.loc[(log['region'] != 'Russia') & (log['traffic_source'] == 'paid'), 'source_type'] = 'other'

log.head(20)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,email


### Домашнее задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [2]:
import re

urls_data = pd.read_csv('URLs.txt')
urls_data.loc[urls_data['url'].str.contains(pat = '/\d{8}-', regex = True)].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


### Домашнее задание 3

В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [3]:
from pymystem3 import Mystem

def creat_lemmas (string):
    kw = Mystem() # Мы тут создаем экземляр класса (объект kw)?
    lemmas = ''
    lemmas = lemmas.join(kw.lemmatize(string)) # А тут применяем метод объекта kw?
    return lemmas[:-1]

data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})

data['lemmas'] = data['keyword'].apply(creat_lemmas)

data.head()

,keyword,shows,lemmas
0,курс гривны к рублю,125076,курс гривна к рубль
1,доллары в рубли,114173,доллар в рубль
2,100 долларов в рублях,97534,100 доллар в рубль
3,курс рубля,53546,курс рубль
